## Linear Regression



While Excel and many Python plotting libraries offer straightforward methods to plot a regression line, conducting a thorough analysis of linear regression is actually quite complex. In this module, we will demonstrate the proper approach to linear regression analysis. Specifically, we will examine whether the data is appropriate for this type of analysis, how to visualize confidence intervals, and, as a valuable addition, how to plot the confidence intervals for predictions derived from the linear regression model.



### Causation versus Correlation



Back in my home country, and before the hippy movement changed our culture, kids, who were curious about where the babies come from, were told that they are brought by the stork (a large bird, see Fig.[ref:fig:storksa](ref:fig:storksa)). Storks were indeed a common sight in rural areas, and large enough to sell this story to a 3-year-old.

![img](Ringedwhitestork.jpg "The Stork. Image by Soloneying, Wikimedia Downloaded Nov 22, 2019.")

As mature scientists who value critical thinking, we seek data to support this idea. Specifically, we expect to find a strong correlation between the number of storks and the number of babies born. Surprisingly, these two variables do indeed correlate in a statistically significant manner. Countries with larger stork populations tend to have higher birthrates. This phenomenon, where both variables increase together, is referred to as a positive correlation.. See Fig. [4](#org78fe66b)

![img](./stork_new.png "The birthrate and the number of stork pairs correlate in a statistically significant way. This analysis suggests that each stork pair delivers about 29 human babies, and that about 225 thousand babies were born otherwise. Data after Matthews 2000.")

This does not prove that storks deliver babies. Clearly, the correlation between two observable quantities does not imply causation. A more plausible explanation is that both variables are influenced by a shared factor, such as industrialization. It is a common mistake to confuse correlation with causation. Another good example is to correlate drinking with heart attacks. This surely will correlate, but the story is more complex. Are there, e.g., patterns like drinkers tend to do less exercise than non-drinkers? So even if you have a good hypothesis why two variables are correlated, the correlation itself proves nothing.

 
Regardless of a causal relationship, the correlation between two datasets can be quantified using the Pearson Product Moment Correlation Coefficient (PPMCC, commonly referred to as r). This coefficient indicates both the strength and direction of the relationship between two variables. The PPMCC ranges from +1 (indicating a perfect positive correlation) to -1 (indicating a perfect negative or inverse correlation). Correlations are considered weak when they fall between +0.3 and -0.3, and strong when they exceed +0.7 or are lower than -0.7. It is important to note that correlation analysis does not assume any specific functional form between the dependent variable (y) and the independent variable (x). This means that the PPMCC does not provide information on whether the correlation is linear, logarithmic, exponential, or of any other form.
We can use the `corr()` method of the pandas series object to calculate the PPMCC:



In [1]:
import pandas as pd  # inport pandas as pd
import pathlib as pl

fn: str = "storks_vs_birth_rate.csv"  # file name
cwd: pl.Path = pl.Path.cwd()
fqfn: pl.Path = pl.Path(f"{cwd}/{fn}")

if not fqfn.exists():
    raise FileNotFoundError(f"Cannot find file {fqfn}")

df: pd.DataFrame = pd.read_csv(fn)  # read data
df.columns = ["Babies", "Storks"]  # replace column names 
b: pd.Series = df["Babies"]
s: pd.Series = df["Storks"]

print(f" r = {s.corr(b):.2f}")

Based on this analysis, we can confirm that the number of babies and storks correlate.



### Understanding the results of a linear regression analysis



Linear regression analysis builds upon correlation analysis by evaluating how effectively a linear function, such as a straight line, represents the relationship between two variables, x and y. A straight line is described by the equation `y = mx + b`, where adjustments are made to the coefficients m (slope) and b (y-intercept) to minimize the difference between the observed data points and the predictions made by the model. Numerous libraries are available to facilitate this fitting process, and ideally, they provide not only the resulting linear model (the equation of the fitted line) but also additional parameters that assess the quality of the fit.

The most common parameter is the coefficient of determination, denoted as r<sup>2</sup>. This metric indicates how effectively a linear relationship can explain the correlation between the dependent variable `y` and a single independent variable `x`. In the figure above, r<sup>2</sup> = 0.38, suggesting that 38% of the variation in the number of new-born babies can be accounted for by a linear correlation with the number of storks. Often, however, multiple independent variables are needed to better capture the variability in the data. In such cases, Multiple Linear Regression is utilized, represented as `y = x1 + x2 + x3… + xn + b`. This approach involves simultaneously solving a system of linear equations (a concept from Linear Algebra) and provides the Multiple Coefficient of Determination, denoted as R<sup>2</sup>, which quantifies the proportion of variability in `y` explained by the combination of independent variables x<sub>i</sub>. Many programs default to using R<sup>2</sup> even with a single independent variable, which can be misleading since capital R<sup>2</sup> should be reserved for analyses that involve multiple independent variables.

From a user perspective, we are interested to understand how good the model is, and how to interpret the key indicators of a given regression model:

-   **r<sup>2</sup>:** or the coefficient of determination. This value is in the range from zero to one and expresses how much of the observed variance in the data is explained by the regression model. So a value of r<sup>2</sup>=0.7 indicates that 70% of the variance is explained by the model, and that 30% of the variance is explained by other processes which are not captured by the linear model (e.g., measurements errors, or some non-linear effect affecting `x` and `y`). In Fig. [BROKEN LINK: fig:storks] 38% of the variance in the birthrate can be explained by the increase in stork pairs.
-   **p:** In linear regression, we hypothesize that `y` depends on `x` and that they are connected by a linear equation. When testing a hypothesis, it is essential to also evaluate the **null hypothesis**, which in this context asserts that `y` is unrelated to `x`. The p-value indicates the probability that the null hypothesis is true. For instance, a p-value of 0.1 suggests a 10% chance that there is no correlation in your data, while a p-value of 0.01 indicates a 1% chance of no correlation. Generally, we can reject the null hypothesis if `p < 0.05`, implying that we are 95% confident the null hypothesis is incorrect. In Fig. [BROKEN LINK: fig:storks], we achieve a confidence level of 99.2% in deeming the null hypothesis false. It is important to note that the relationship between r² and p is not always straightforward.



### Testing the data



#### Histograms




The Storks data set is interesting; however, it does not meet a crucial requirement for linear regression analysis: the data must exhibit a [Gaussian Normal Distribution](https://en.wikipedia.org/wiki/Normal_distribution) (or bell curve). 
To quickly assess the distribution of your data, you can create a [histogram plot](https://en.wikipedia.org/wiki/Histogram) to determine whether the x and y values are normally distributed.



In [1]:
import matplotlib.pyplot as plt
import pandas as pd  # import pandas as pd
import pathlib as pl

fn: str = "storks_vs_birth_rate.csv"  # file name
cwd: pl.Path = pl.Path.cwd()
fqfn: pl.Path = pl.Path(f"{cwd}/{fn}")
if not fqfn.exists():  # check if the file is actually there
    raise FileNotFoundError(f"Cannot find file {fqfn}")

df: pd.DataFrame = pd.read_csv(fn)  # read data
df.columns = ["Babies", "Storks"]  # replace colum names
display(df.head())  # test that all went well

fig, [ax1, ax2] = plt.subplots(nrows=2, ncols=1)  #
ax1.hist(
    df.iloc[:, 0],
)
ax2.hist(df.iloc[:, 1])
ax1.set_title("Babies")
ax2.set_title("Storks")
plt.tight_layout()
plt.savefig("strk_histogram.png")
plt.show()

|   | Babies | Storks |
|---+--------+--------|
| 0 |    118 |      1 |
| 1 |    188 |      4 |
| 2 |    551 |      5 |
| 3 |     59 |      9 |
| 4 |     83 |    100 |

![img](strk_histogram.png)
As shown in the histogram, our data clearly does not follow a normal distribution. Nevertheless, we will proceed with it, as it is an interesting dataset. However, conducting the aforementioned test is essential if you plan to perform a thorough regression analysis in the future!



#### Qqplots




Another method to assess normality is by using a [QQ plot](https://en.wikipedia.org/wiki/Q%E2%80%93Q_plot), which determines if your data follows a normal distribution. However, the statsmodels QQ plot function does not adhere to the standard matplotlib syntax, making it challenging to customize colors. 



In [1]:
import statsmodels.api as sm

fig, ax = plt.subplots()
sm.qqplot(df['Babies'], ax=ax, line="s")
plt.savefig("qqplot.png")
plt.show()

![img](qqplot.png)
Here's how to interpret a QQ plot:

1.  **Axes**: 
    -   The x-axis represents the theoretical quantiles from the expected distribution (often a normal distribution).
    -   The y-axis represents the quantiles from the observed dataset.

2.  **Points**: 
    -   Each point on the plot corresponds to a quantile of the observed data plotted against a quantile from the theoretical distribution.
    -   If the dataset follows the theoretical distribution closely, the points will lie along a straight line (often the 45-degree line).

3.  **Line of Identity**: 
    -   A reference line (y = x) is typically drawn, representing where points would lie if the two distributions were identical.
    -   Deviations from this line indicate differences between the two distributions.

4.  **Interpreting Patterns**:
    -   **Straight Line**: If the points lie close to the line, it suggests that the observed data follows the theoretical distribution well.
    -   **Curve or S-Curve**: If the points curve away from the line (upward or downward), it indicates that the tails of the observed distribution are heavier or lighter than the theoretical distribution.
    -   **Outliers**: Points that are far from the line can indicate potential outliers in the data.

5.  **Specific Cases**:
    -   **Left Skewed Distribution**: Points tend to be above the line for lower quantiles and below for higher quantiles.
    -   **Right Skewed Distribution**: Points tend to be below the line for lower quantiles and above for higher quantiles.
    -   **Normal Distribution**: For normally distributed data, the QQ plot should closely resemble a straight line.

By analyzing these aspects, you can see that the stork data is not normal distributed.



#### The Shapiro-Wilk Test



The tests mentioned above depend on visual interpretation. The [Shapiro-Wilk test](https://en.wikipedia.org/wiki/Shapiro%E2%80%93Wilk_test) offers a systematic approach to assess normal distribution. The following snippet illustrates how to implement and interpret the test.



In [1]:
from scipy.stats import shapiro

# Do a Shapiro Wilk test
stat, p = shapiro(df['Storks'])
print(f"Shapiro Wilk Test Statistics W = {stat:.3f}, p = {p:.3f}")

# interpretation
if p > 0.05 and stat > 0.8:
    print("Sample looks Gaussian")
elif p <= 0.05 and stat > 0.8:
    print("Sample might be Gaussian")
elif p <= 0.05 and stat <= 0.8:
    print("Sample likely not Gaussian")

Shapiro Wilk Test Statistics W = 0.615, p = 0.000
Sample likely not Gaussian

Here's how to interpret the results of the test:

1.  **Null Hypothesis (H0)**: The null hypothesis states that the data follows a normal distribution.

2.  **Alternative Hypothesis (H1)**: The alternative hypothesis states that the data does not follow a normal distribution.

3.  **Test Statistic (W)**: The test calculates a test statistic (W) that compares the observed distribution of the data to a normal distribution. A W value close to 1 indicates that the data may be normally distributed.

4.  **p-value**: The p-value is a crucial output of the Shapiro-Wilk test. It indicates the probability of observing the test statistic under the null hypothesis.
    -   A **high p-value** (typically greater than 0.05) suggests that you fail to reject the null hypothesis, implying that there is not enough evidence to conclude that the data is not normally distributed.
    -   A **low p-value** (typically less than or equal to 0.05) suggests that you reject the null hypothesis, indicating that the data does not follow a normal distribution.

5.  **Sample Size Considerations**: 
    -   The Shapiro-Wilk test is more powerful with small sample sizes. With larger samples, even minor deviations from normality can lead to significant p-values.

6.  **Practical Implications**: If your dataset is determined to be significantly different from normal (p-value less than 0.05), you may consider using non-parametric statistical methods for further analysis, as many statistical tests assume normality.

In summary, the key to interpreting the Shapiro-Wilk test lies in examining the p-value in relation to your chosen significance level (commonly 0.05), which helps you determine whether your data can be considered normally distributed or not.



### The statsmodels library



Python's success largely stems from its vast array of third-party libraries, which are generally free to use, in contrast to Matlab. In this section, we will utilize the "statsmodels" library, though many other statistical libraries are also available. The statsmodels library offers several interfaces, and we will focus on the formula interface, which resembles R's formula syntax (for more details, see [https://www.statsmodels.org/dev/example_formulas.html](https://www.statsmodels.org/dev/example_formulas.html)). However, it's important to note that not all functions in statsmodels are accessible through this interface at this time.

For the regression model, we aim to investigate whether the number of storks correlates with the number of babies born. In other words, does the birth rate depend on the stork population? To do this, we must define a statistical model and evaluate how well its predictions align with the data. **It is important to note that as of March 2025, the statsmodels library does not support type hinting.** 

To enhance clarity, we will replace the specific data-dependent names in the lengthy code with more general aliases. For the independent data ("Storks"), we will use `X`, and for the dependent data ("Babies"), we will use `Y`. I  use the `df.columns()` method (line 12) to set the column names to `"X"` and `"Y`. This approach allows us to avoid having to change variable names in multiple locations if we repeat this process in the future. After importing the data, we proceed to create a statistical model on line 16 of the code below. Note how the model is specified using the formula `"Y" ~ "X"` indicating that the number of Babies (`Y`)is dependent on the number of Storks (`X`). It is essential that these names match the variable names in the data frame `df`. Once the model is defined, we call the `fit()` method to fit the model using the data (line 17). The results of this method will be stored in the `results` variable. Line 18 subsequently invokes the `summary()` method on the results object.



In [1]:
import statsmodels.formula.api as smf
import pandas as pd  # import pandas as pd
import pathlib as pl

fn: str = "storks_vs_birth_rate.csv"  # file name
cwd: pl.Path = pl.Path.cwd()
fqfn: pl.Path = pl.Path(f"{cwd}/{fn}")
if not fqfn.exists():  # check if the file is actually there
    raise FileNotFoundError(f"Cannot find file {fqfn}")

df: pd.DataFrame = pd.read_csv(fn)  # read data
df.columns = ["Y", "X"]  # replace colum names

model = smf.ols(formula="Y ~ X", data=df) # create model
results = model.fit()  # fit the model to the data
display(results.summary())  # print the results of the analysis

Plenty of information here, probably more than you asked for. Let's tease out the important ones:

-   The first line states that `Babies` is the dependent variable. This is useful and will help you to catch errors in your model definition.
-   The second line confirms that this is an ordinary least squares model
-   Then, there are also a couple of warnings, indicating that your data quality may be less than excellent. But we knew this already from testing whether the data is normal distributed or not.

If you compare the output with Figure [ref:fig:storks](ref:fig:storks), you can see that r<sup>2</sup> value is called "R-squared", the p-value is called "Prob (F-statistic)", the y-intercept is the first value in the "Intercept" row, the slope is the first value in the "Storks" row. You can also extract these parameters from the model results object like this:



In [1]:
""" Retrieve parameters from the model results. Note
that the dictionary key 'x' must be equal to the
name of the independent variable used in the model
definition (i.e., y~x)
"""

slope: float = results.params["X"]  # the slope
y_0: float = results.params["Intercept"]  # the y-intercept
r_square: float = results.rsquared  # rsquare
p_value: float = results.pvalues["X"]  # the p-value

### Adding the regression line and confidence intervals



The r<sup>2</sup> and p-value give us some indication of how well our regression model performs. However, we can add further information to our graph:

-   The line which represents the regression model
-   The confidence intervals that indicate the confidence we have in our
    regression model.
-   The confidence intervals that indicate the confidence we have in the
    predictions we make based on our regression model



#### Accessing the confidence interval data



Confidence intervals visually illustrate the reliability of your linear model. There are two types of confidence intervals: A) the confidence that your linear model accurately represents the data—meaning that any regression line within this interval is likely valid; and B) the confidence in the predictions derived from your model. This second interval is often more intriguing, yet commercial software that offers this information is scarce. A crucial parameter related to confidence intervals is the level of significance . This level indicates how confident you wish to be, such as 99%, 95%, or 70% certain that you are right. Statistical software typically designates the significance level with the alpha keyword, and it is customary to express this value as 1 minus the significance (for example, &alpha; = 0.05 corresponds to a 95% confidence level).

We can retrieve this information from our model results object in the following way:



In [1]:
prediction_data = results.get_prediction(df.X) # extract prediction data
pred_summary = pred.summary_frame(alpha=0.05) # get summary at 95%

# get the confidence interval for the model
model_ci_low: NDArrayFloat = pred_summary['mean_ci_lower']
model_ci_upp: NDArrayFloat = pred_summary['mean_ci_upper']

# get the confidence interval for the prediction
predict_ci_low: NDArrayFloat = pred_summary['obs_ci_lower']
predict_ci_upp: NDArrayFloat = pred_summary['obs_ci_upper']

#### Plotting the confidence interval data



The upper and lower confidence boundaries describe the upper and lower boundaries of an area. We can either plot these boundaries as a line plot or as a shaded area.  Matplotlib provides the `fill_between` method @@latex:\index{matplotlib!fill-between}to shade the area between two lines:



In [1]:
ax.fill_between(storks, model_ci_low, model_ci_up, alpha=0.1, color="C1")

Note the use of the `alpha` keyword. This has nothing to do with the alpha which is used in statistics. Rather, it describes the transparency of the object you are drawing. If you set it to one (the default), the object will be fully opaque. If you set it to zero, it will be fully transparent (so you won't see it).  See the code below for an actual example.



### Creating the Stork Figure



Now let's put it all together. Note that when we draw the figure, it matters whether we draw the confidence intervals first or last. Change the order in the code below, to see the difference.



In [1]:
import numpy as np
import numpy.typing as npt
import pandas as pd
import matplotlib.pyplot as plt
import pathlib as pl
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import summary_table

fn: str = "storks_vs_birth_rate.csv"  # data file name
fig_name: str = "stork_new.png"
x_axis_label = "Stork Pairs"
y_axis_label = "Newborn Babies [$10^3$]"
significance = 0.05  # = 1 - sig > 0.95 = 95% significance

cwd: pl.Path = pl.Path.cwd()
fqfn: pl.Path = pl.Path(f"{cwd}/{fn}")
if not fqfn.exists():  # check if the file is actually there
    raise FileNotFoundError(f"Cannot find file {fqfn}")

df: pd.DataFrame = pd.read_csv(fn).dropna()  # read data
df.columns = ["Y", "X"]  # replace colum names
df.sort_values(by = "X", ascending=True)

# ------ create linear regression model ------
model: smf.ols = smf.ols(formula="Y ~ X", data=df)
results: model.fit = model.fit()  # fit the model to the data

# ------ extract model parameters
fitted_values: NDArrayFloat = results.fittedvalues
slope: float = results.params["X"]  # the slope = name of y
y_0: float = results.params["Intercept"]  # the y-intercept
r_square: float = results.rsquared  # r_square
p_value: float = results.pvalues["X"]  # the p_value for y
ds: str = (
    f"y = {y_0:1.4f}+x*{slope:1.4f}\n"
    f"$r^2$ = {r_square:1.2f}\n"
    f"p = {p_value:1.4f}"
)

# ------ extract confidence intervals ------
prediction_data = results.get_prediction(df.X) # extract prediction data
pred_summary = pred.summary_frame(alpha=significance) # get summary at 95%

# get the confidence interval for the model
model_ci_low: NDArrayFloat = pred_summary['mean_ci_lower']
model_ci_upp: NDArrayFloat = pred_summary['mean_ci_upper']

# get the confidence interval for the prediction
predict_ci_low: NDArrayFloat = pred_summary['obs_ci_lower']
predict_ci_upp: NDArrayFloat = pred_summary['obs_ci_upper']
NDArrayFloat = npt.NDArray[np.float64]

# ------ create plot ------
fig, ax = plt.subplots()  # create canvas and axis objects

# plot confidence intervals first
ax.fill_between(df.X, predict_ci_low, predict_ci_upp, alpha=0.1, color="C1")
ax.fill_between(df.X, model_ci_low, model_ci_upp, alpha=0.2, color="C1")
# add data points
ax.scatter(df.X, df.Y, color="C0")
# regression line
ax.plot(df.X, fitted_values, color="C1")
# plot options and annotations
plt.style.use("ggplot")
fig.set_size_inches(6, 4)
fig.set_dpi(120)
ax.text(1000, 1750, ds, verticalalignment="top")
ax.set_xlabel(x_axis_label)
ax.set_ylabel(y_axis_label)
fig.set_tight_layout("tight")
fig.savefig(fig_name)
plt.show()

### References



-   Robert Matthews, 2000. Storks Devilver Babies (p = 0.008), Teaching Statistics, vol22:2, p. 36-38, [https://doi.org/10.1111/1467-9639.00013](https://doi.org/10.1111/1467-9639.00013),

